In [ ]:
# %load ../env.py
%reload_ext autoreload
%autoreload 2
%pylab inline
%matplotlib inline

import matplotlib.text as text
import pandas as pd
import numpy as np
import pylab as pl
import scipy as sp
import sys
import rpy2 
import os 

#Set environment variables


# Set up the local source files
TOP = '/'.join(os.getcwd().split('/')[:-2])+'/'
#TOP = "/share/home/ishah/ipynb/pb/HTTR_WF/"

LIB = TOP+'lib'
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

os.environ['PYTHONPATH']=LIB


DAT_DIR = TOP + '/data/'
FIG_DIR = TOP + '/figs/'

if not os.path.exists(DAT_DIR): os.mkdir(DAT_DIR)
if not os.path.exists(FIG_DIR): os.mkdir(FIG_DIR)


from db.mongo import *


# Summary

Create the new version of the GenRA database. The last version is genra_v3 so this will be genra_dev_v4. The keys steps are:

* Chemical data: load chemical structure data from DSSTox and create chemical fingerprints
* Bioactivity data: load bioactivity data from different sources (ToxCast, Tox21, other?) and create bioactivity fingerprints
* Toxicity data: load toxicity data from toxrefdb and create toxicity fingerprints
* Physprop data: load physprob data from DSSTox

# Chemical Data

## Loading from DSSTox
Todo: describe how the chemical information is loaded into the database and kept updated. 

Right now just copy over collection that is already setup

`mongodump -j 10 -d ghelman -c compounds -o dump`

`mongodump -j 10 -d ghelman -c logs -o dump`

`mongorestore -j 10 -d genra_dev_v4 -c compounds dump/compounds.bson`

`mongorestore -j 10 -d genra_dev_v4 -c logs dump/logs.bson`


In [ ]:
DB = openMongo(db='genra_dev_v4')
GH = openMongo(db='ghelman')
GH.collection_names()

In [ ]:
DB.compounds.find_one({'name':'Bisphenol A'},dict(synonyms=0))

In [ ]:
DB.logs.find_one(dict(dsstox_cid='DTXCID30182'))

In [ ]:
for C in DB.compounds.find({},dict(_id=0,synonyms=0),no_cursor_timeout=True):
    L = DB.logs.update_one(dict(dsstox_cid=C['dsstox_cid']),{'$set':{'dsstox_sid':C['dsstox_sid']}})
